# Medical question and answer

In [10]:
# !pip install peft
# !pip install datasets
# !pip install --upgrade transformers
# !pip install evaluate
# !pip install rouge_score
# !pip install tensorboard

# Preparing the Dataset.

In [11]:
from datasets import load_dataset, DatasetDict

# Summarization
data_summarization = load_dataset("json", data_files="./src/data/medical_meadow_mediqa.json")

def split_dataset(data):
  # Perform a train-test split
  data = data["train"].train_test_split(test_size=0.1)

  # Further split the training data into training and validation sets
  test_data = data["test"]
  train_data = data["train"].train_test_split(test_size=0.1)
  valid_data = train_data["test"]
  train_data = train_data["train"]
  # Print dataset splits
  print("Train data:", len(train_data))
  print("Validation data:", len(valid_data))
  print("Test data:", len(test_data))

  return train_data, valid_data, test_data

# Data Wiki
train_data_patient, valid_data_patient, test_data_patient = split_dataset(data_summarization)
data_patient_wiki = DatasetDict({
    "train": train_data_patient,
    "validation": valid_data_patient,
    "test": test_data_patient
})


Train data: 1788
Validation data: 199
Test data: 221


In [12]:
print(data_patient_wiki)

print(data_patient_wiki['train'][10])

print(data_patient_wiki['train'][10]['input'])
print(data_patient_wiki['train'][10]['output'])

DatasetDict({
    train: Dataset({
        features: ['instruction', 'input', 'output'],
        num_rows: 1788
    })
    validation: Dataset({
        features: ['instruction', 'input', 'output'],
        num_rows: 199
    })
    test: Dataset({
        features: ['instruction', 'input', 'output'],
        num_rows: 221
    })
})
{'instruction': 'how do you get pulmonary hypertension', 'input': " Pulmonary hypertension Overview Pulmonary hypertension is a type of high blood pressure that affects the arteries in your lungs and the right side of your heart. In one form of pulmonary hypertension, tiny arteries in your lungs, called pulmonary arterioles, and capillaries become narrowed, blocked or destroyed. This makes it harder for blood to flow through your lungs, and raises pressure within your lungs' arteries. As the pressure builds, your heart's lower right chamber (right ventricle) must work harder to pump blood through your lungs, eventually causing your heart muscle to weaken and

In [13]:
from datasets import Dataset


prefix = "summarize: "


def tokenization_dataset(dataset, tokenizer, max_length=512):
    # Tokenize the dataset
    def tokenize_function(examples):
        inputs = [prefix + doc for doc in examples["input"]]
        model_inputs = tokenizer(inputs, max_length=1024, truncation=True)

        labels = tokenizer(text_target=examples["output"], max_length=128, truncation=True)

        model_inputs["labels"] = labels["input_ids"]
        return model_inputs

    # Apply the tokenization to train, validation, and test sets
    train_data = dataset["train"].map(tokenize_function, batched=True)
    valid_data = dataset["validation"].map(tokenize_function, batched=True)
    test_data = dataset["test"].map(tokenize_function, batched=True)

    return train_data, valid_data, test_data

# Modeling and representation

In [14]:
from transformers import (AutoModelForQuestionAnswering,
                          TrainingArguments,
                          Trainer,
                          AutoTokenizer,
                          GPT2ForQuestionAnswering,
                          T5ForQuestionAnswering)

from dotenv import load_dotenv
import os
# Load environment variables from the .env file
load_dotenv()

checkpoint="google-t5/t5-small"
import os
T5Tokenizer = AutoTokenizer.from_pretrained(checkpoint, token=os.getenv("HF_TOKEN"))
#T5QA = T5ForQuestionAnswering.from_pretrained("t5-base")

from transformers import DataCollatorForSeq2Seq
data_collator = DataCollatorForSeq2Seq(tokenizer=T5Tokenizer, model=checkpoint)

from transformers import AutoModelForSeq2SeqLM, Seq2SeqTrainingArguments, Seq2SeqTrainer
model = AutoModelForSeq2SeqLM.from_pretrained(checkpoint, token=os.getenv("HF_TOKEN"))

import evaluate

rouge = evaluate.load("rouge")

In [15]:
train_data_T5, valid_data_T5, test_data_T5 = tokenization_dataset(data_patient_wiki, T5Tokenizer)

Map:   0%|          | 0/1788 [00:00<?, ? examples/s]

Map:   0%|          | 0/199 [00:00<?, ? examples/s]

Map:   0%|          | 0/221 [00:00<?, ? examples/s]

# Config Args.


TRANSFORMERS_MODELS_TO_LORA_TARGET_MODULES_MAPPING = {
    "t5": ["q", "v"],
    "mt5": ["q", "v"],
    "bart": ["q_proj", "v_proj"],
    "gpt2": ["c_attn"],
    "bloom": ["query_key_value"],
    "blip-2": ["q", "v", "q_proj", "v_proj"],
    "opt": ["q_proj", "v_proj"],
    "gptj": ["q_proj", "v_proj"],
    "gpt_neox": ["query_key_value"],
    "gpt_neo": ["q_proj", "v_proj"],
    "bert": ["query", "value"],
    "roberta": ["query", "value"],
    "xlm-roberta": ["query", "value"],
    "electra": ["query", "value"],
    "deberta-v2": ["query_proj", "value_proj"],
    "deberta": ["in_proj"],
    "layoutlm": ["query", "value"],
    "llama": ["q_proj", "v_proj"],
    "chatglm": ["query_key_value"],
    "gpt_bigcode": ["c_attn"],
    "mpt": ["Wqkv"],
}

In [22]:
import peft
from peft import LoraConfig, get_peft_model, PeftModel

def define_model(model, lora_config):
    peft_model = get_peft_model(model, lora_config)
    print(peft_model.print_trainable_parameters())
    return peft_model

lora_config_T5 = LoraConfig(
        r=4, #As bigger the R bigger the parameters to train.
        lora_alpha=1, # a scaling factor that adjusts the magnitude of the weight matrix. Usually set to 1
        target_modules=["q", "v"], #You can obtain a list of target modules in the URL above.
        lora_dropout=0.05, #Helps to avoid Overfitting.
        bias="lora_only", # this specifies if the bias parameter should be trained.
        task_type="SEQ_2_SEQ_LM"
    )

peft_T5 = define_model(model, lora_config_T5)

import evaluate
import numpy as np

# Load relevant summarization metrics
rouge_metric = evaluate.load("rouge")
bleu_metric = evaluate.load("bleu")

def compute_metrics(p, tokenizer):
    print("Computing metrics...")
    predictions, labels = p
    # If predictions is a t uple, get the logits
    if isinstance(predictions, tuple):
        predictions = predictions[0]
    # Replace -100 in labels as they are ignored in loss but we want to decode them
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)

    # Get the predicted token ids (if not already decoded)
    pred_ids = np.argmax(predictions, axis=-1)

    # Decode the predictions and labels
    pred_texts = tokenizer.batch_decode(pred_ids, skip_special_tokens=True)
    ref_texts = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # Print example output for debugging
    print("Example Prediction:")
    print("Predicted Summary:", pred_texts[0])
    print("Reference Summary:", ref_texts[0])
    print()

    # Compute ROUGE scores
    rouge_results = rouge_metric.compute(predictions=pred_texts, references=ref_texts, use_stemmer=True)

    # Compute BLEU score (expects references as list of lists)
    bleu_results = bleu_metric.compute(predictions=pred_texts, references=[[ref] for ref in ref_texts])

    return {
        "rouge1": rouge_results["rouge1"],
        "rouge2": rouge_results["rouge2"],
        "rougeL": rouge_results["rougeL"],
        "rougeLsum": rouge_results["rougeLsum"],
        "bleu": bleu_results["bleu"],
        "Predicted Summary": pred_texts[0],
        "Reference Summary": ref_texts[0],
    }


trainable params: 147,456 || all params: 60,654,080 || trainable%: 0.2431
None


# Model definition

In [ ]:
import json
class DataTrainer:
    def __init__(self, model, dataset_train, dataset_valid, tokenizer, data_collator):
        self.model = model
        self.dataset_train = dataset_train
        self.dataset_valid = dataset_valid
        self.tokenizer = tokenizer
        self.data_collator = data_collator
        self.training_args = TrainingArguments(
            output_dir="results",
            # eval_strategy="epoch",
            eval_strategy="steps",
            eval_steps=100,
            learning_rate=3e-2,
            auto_find_batch_size=True,
            per_device_train_batch_size=2,
            per_device_eval_batch_size=1,
            num_train_epochs=3,
            weight_decay=0.01,
            gradient_accumulation_steps=4,
            push_to_hub=False,
            # save_strategy="epoch",
            save_strategy="steps",
            save_steps=200,
            report_to="tensorboard",
            logging_dir="logs",
            load_best_model_at_end=True,
            fp16=True, 
        )

        self.trainer = Trainer(
            model=self.model,
            args=self.training_args,
            train_dataset=self.dataset_train,
            eval_dataset=self.dataset_valid,
            tokenizer=self.tokenizer,
            data_collator=self.data_collator,
            compute_metrics=lambda p: self.compute_and_log_metrics(p, self.tokenizer),
        )

    def compute_and_log_metrics(self, p, tokenizer):
        metrics = compute_metrics(p, tokenizer)
        os.makedirs("custom_metrics", exist_ok=True)
        with open("custom_metrics/metrics.json", "a") as f:
            json.dump(metrics, f, indent=2)
        return metrics

    def train(self):
        print(f"Model is on device: {next(self.model.parameters()).device}")
        print("Starting training...")
        # Check for last checkpoint
        last_checkpoint = None
        if os.path.isdir(self.output_dir):
            checkpoints = [os.path.join(self.output_dir, d) for d in os.listdir(self.output_dir)
                           if os.path.isdir(os.path.join(self.output_dir, d)) and d.startswith("checkpoint")]
            if checkpoints:
                last_checkpoint = max(checkpoints, key=os.path.getmtime)
                print(f"Resuming from checkpoint: {last_checkpoint}")
        
        self.trainer.train(resume_from_checkpoint=last_checkpoint)

# Train

In [28]:
trainer_distilBert = DataTrainer(
    model=peft_T5,
    dataset_train=train_data_T5,
    dataset_valid=valid_data_T5,
    tokenizer=T5Tokenizer,
    data_collator=data_collator
)
trainer_distilBert.train()

Model is on device: cuda:0
Starting training...


  0%|          | 0/669 [00:00<?, ?it/s]

  0%|          | 0/199 [00:00<?, ?it/s]

Computing metrics...
Example Prediction:
Predicted Summary: Theeander poisoning occurs when someone eats the flowers or chews the leaves or stems of the oleander plant (erium oleander or its relative, the yellow oleander (ascabela thevetia) This you or someone you are with has has exposure, call your local emergency number (such as 911), or your local poison center can be reached directly by calling the national toll-free Poison Help hotline (1-800-222-1222) from anywhere in the United States. can call if you have any questions about poisoning or
Reference Summary: Oleander poisoning occurs when someone eats the flowers or chews the leaves or stems of the oleander plant (Nerium oleander), or its relative, the yellow oleander (Cascabela thevetia). If you or someone you are with has an exposure, call your local emergency number (such as 911), or your local poison center can be reached directly by calling the national toll-free Poison Help hotline (1-800-222-1222) from anywhere in the Uni

Trainer is attempting to log a value of "Theeander poisoning occurs when someone eats the flowers or chews the leaves or stems of the oleander plant (erium oleander or its relative, the yellow oleander (ascabela thevetia) This you or someone you are with has has exposure, call your local emergency number (such as 911), or your local poison center can be reached directly by calling the national toll-free Poison Help hotline (1-800-222-1222) from anywhere in the United States. can call if you have any questions about poisoning or" of type <class 'str'> for key "eval/Predicted Summary" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "Oleander poisoning occurs when someone eats the flowers or chews the leaves or stems of the oleander plant (Nerium oleander), or its relative, the yellow oleander (Cascabela thevetia). If you or someone you are with has an exposure, call your local emergency n

...

{'eval_loss': 1.0830398797988892, 'eval_rouge1': 0.7919926106547108, 'eval_rouge2': 0.6565430589861576, 'eval_rougeL': 0.7622512015102045, 'eval_rougeLsum': 0.763741829142786, 'eval_bleu': 0.6518930812536841, 'eval_Predicted Summary': 'Oleander poisoning occurs when someone eats the flowers or chews the leaves or stems of the oleander plant (erium oleander or its relative, the yellow oleander (ascabela thevetia This you or someone you are with has an exposure, call your local emergency number (such as 911), or your local poison center can be reached directly by calling the national toll-free Poison Help hotline (1-800-222-1222) from anywhere in the United States. should call forif you have any questions about poisoning or poison', 'eval_Reference Summary': 'Oleander poisoning occurs when someone eats the flowers or chews the leaves or stems of the oleander plant (Nerium oleander), or its relative, the yellow oleander (Cascabela thevetia). If you or someone you are with has an exposure,

  0%|          | 0/199 [00:00<?, ?it/s]

Computing metrics...
Example Prediction:
Predicted Summary: Theeander poisoning occurs when someone eats the flowers or chews the leaves or stems of the oleander plant (erium oleander or its relative, the yellow oleander (ascabela thevetia This you or someone you are with has an exposure, call your local emergency number (such as 911), or your local poison center can be reached directly by calling the national toll-free Poison Help hotline (1-800-222-1222) from anywhere in the United States. This should call forif you have any questions about poisoning or poison
Reference Summary: Oleander poisoning occurs when someone eats the flowers or chews the leaves or stems of the oleander plant (Nerium oleander), or its relative, the yellow oleander (Cascabela thevetia). If you or someone you are with has an exposure, call your local emergency number (such as 911), or your local poison center can be reached directly by calling the national toll-free Poison Help hotline (1-800-222-1222) from any

Trainer is attempting to log a value of "Theeander poisoning occurs when someone eats the flowers or chews the leaves or stems of the oleander plant (erium oleander or its relative, the yellow oleander (ascabela thevetia This you or someone you are with has an exposure, call your local emergency number (such as 911), or your local poison center can be reached directly by calling the national toll-free Poison Help hotline (1-800-222-1222) from anywhere in the United States. This should call forif you have any questions about poisoning or poison" of type <class 'str'> for key "eval/Predicted Summary" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "Oleander poisoning occurs when someone eats the flowers or chews the leaves or stems of the oleander plant (Nerium oleander), or its relative, the yellow oleander (Cascabela thevetia). If you or someone you are with has an exposure, call your l

{'eval_loss': 1.0621877908706665, 'eval_rouge1': 0.7935702995091988, 'eval_rouge2': 0.6603063608093724, 'eval_rougeL': 0.7649514923609717, 'eval_rougeLsum': 0.7662589853777402, 'eval_bleu': 0.6556354522872686, 'eval_Predicted Summary': 'Theeander poisoning occurs when someone eats the flowers or chews the leaves or stems of the oleander plant (erium oleander or its relative, the yellow oleander (ascabela thevetia This you or someone you are with has an exposure, call your local emergency number (such as 911), or your local poison center can be reached directly by calling the national toll-free Poison Help hotline (1-800-222-1222) from anywhere in the United States. This should call forif you have any questions about poisoning or poison', 'eval_Reference Summary': 'Oleander poisoning occurs when someone eats the flowers or chews the leaves or stems of the oleander plant (Nerium oleander), or its relative, the yellow oleander (Cascabela thevetia). If you or someone you are with has an ex